In [ ]:
# Portal
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import pandas as pd
import os
import handcalcs.render
import forallpeople
from IPython.display import HTML, display, Math, Latex, SVG
from handcalcs.decorator import handcalc
from IPython.display import Markdown as md
from math import sqrt, pi, ceil
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
from collections import defaultdict
forallpeople.environment("structural",top_level=True)
from selenium.webdriver.chrome.options import Options
from math import cos,tan, pi

class portalmethod:
    def __init__(
        self, Height, Storey, Bay, Bay_abc, Bay_123, Force_list, No_frames, folder_name,fy,color
    ) -> None:
        self.height = Height
        self.storey = Storey
        self.bay_abc = Bay_abc
        self.bay_123 = Bay_123
        self.force_list = Force_list
        self.bay_abc_cm = np.hstack((0, Bay_abc.cumsum()))
        self.bay_123_cm = np.hstack((0, Bay_123.cumsum()))
        self.h_line_abc = Storey * Bay_abc
        self.v_line_abc = (Bay_abc + 1) * Storey
        self.frame_face = ["abc", "123"]
        self.no_frames = No_frames
        self.bay = Bay
        self.folder_name = folder_name
        self.load_types = ["Dead", "Live"]
        self.heights = Height * np.ones(Storey)
        self.heights_cm = np.hstack((0, self.heights.cumsum()))
        self.h_line_123 = Storey * Bay_123
        self.v_line_123 = (Bay_123 + 1) * Storey
        self.struc = 3
        self.live_load = 2.4
        self.beam_w = 39.15 / 23
        self.slab_w = 4
        self.wall_w = 9.9
        self.parapits_w = 4

        # self.live_load = 1.6
        # self.beam_w = 39.15 / 23
        # self.slab_w = 1
        # self.wall_w = 3
        # self.parapits_w = 3
        self.latex = []
        self.data = defaultdict(list)
        self.data1 = defaultdict(list)
        self.td = defaultdict(list)
        self.tr = defaultdict(list)
        self.table = defaultdict(list)
        self.path_data = self.create_folder("Data")
        self.path_approximate = self.create_folder("Approximate_Method")
        self.path_latex_comp = self.create_folder("Latex_Comp")
        self.f_y = fy
        self.name_beams = []
        self.column_counter = []
        self.color = color
    def create_folder(self, sub_folder):
        path = os.getcwd() + f"\\{self.folder_name}\\{sub_folder}"
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)
        return path

    def portal_computations(self):
        bay = self.bay
        storey = self.storey

        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
            else:
                bay_widths = self.bay_123
            shear_unit = self.force_list[::-1].cumsum() / (bay * 2)
            csf = np.zeros((self.storey, self.bay + 1))
            csf_shape = np.shape(csf)  # shear force for column
            cmf = np.zeros((self.storey, self.bay + 1))  # Bending moment for column
            bmf = np.zeros((self.storey, self.bay))  # Bending moment for beam
            bsf = np.zeros((self.storey, self.bay))  # Bending moment for beam
            caf = np.zeros((self.storey, self.bay + 1))  # Bending moment for column
            for i in range(storey):
                for j in range(bay + 1):
                    if j == 0:
                        csf[i, j] = np.round(shear_unit[i], 2)
                    elif j == 2:
                        csf[i, j] = np.round(shear_unit[i], 2)
                    else:
                        csf[i, j] = np.round(shear_unit[i] * 2, 2)

            for i in range(storey):
                for j in range(bay + 1):
                    mnf = csf[::-1][i, j] * self.height[i] / 2  # Moment Diagram beam
                    mnf = np.round(mnf, 2)
                    cmf[i, j] = mnf
            for i in range(storey):
                for j in range(bay + 1):
                    if j != bay:
                        bmt = (
                            (cmf[i, 0] + cmf[i + 1, 0])
                            if i < self.storey - 1
                            else cmf[i, 0]
                        )
                        bmt = np.round(bmt, 2)
                        bmf[i, j] = bmt

                        sf = np.round(
                            bmf[i, j] / (bay_widths[j] * 0.5), 2
                        )  # Shear force
                        bsf[i, j] = sf
                    axial_ = 0
                    for i_ in range(i, self.storey):
                        if j == 0:
                            axial_ += bsf[i_, j]
                        elif j == bay:
                            axial_ += bsf[i_, j - 1]
                        else:
                            axial_ += np.round(bsf[i_, j - 1] - bsf[i_, j], 2)
                    axial_ = np.round(axial_, 2)
                    caf[i, j] = axial_
            path = self.create_folder("Data")

            pd.DataFrame(csf).to_csv(f"{path}\\csf-{frame_face}.csv", index=False)
            pd.DataFrame(cmf).to_csv(f"{path}\\cmt-{frame_face}.csv", index=False)
            pd.DataFrame(bmf).to_csv(f"{path}\\bmt-{frame_face}.csv", index=False)
            pd.DataFrame(bsf).to_csv(f"{path}\\bsd-{frame_face}.csv", index=False)
            pd.DataFrame(caf).to_csv(
                f"{path}\\axial_list-{frame_face}.csv", index=False
            )
        return csf, cmf, bmf, bsf, caf

    def frame(self, title,bay_widths,bay_widths_cm):
        # for frame_face in self.frame_face:
        #     if frame_face == "abc":
        #         bay_widths = self.bay_abc
        #         bay_widths_cm = self.bay_abc_cm
        #     elif frame_face == "123":
        #         bay_widths = self.bay_123
        #         bay_widths_cm = self.bay_123_cm
        storey = self.storey
        bay = self.bay
        height = self.height
        heights_cm =self.heights_cm
        force_list = self.force_list
        l = -2
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(1, 1, 1)
        plt.title(label=title, loc="left", fontsize=16, color="green")
        for i in range(storey - 1, -1, -1):
                for j in range(self.bay + 1):
                    if j != bay:
                        ax.plot(
                            [bay_widths_cm[j], bay_widths_cm[j + 1]],
                            [heights_cm[i + 1], heights_cm[i + 1]],
                            color="blue",
                        )  # Horizontal line

                    ax.plot(
                        [bay_widths_cm[j], bay_widths_cm[j]],
                        [heights_cm[i], heights_cm[i + 1]],
                        color="blue",
                    )
                arl = 3
                ax.annotate(
                    str(force_list[i]),
                    xytext=[-arl, heights_cm[i + 1] - 0.1],
                    xy=[0, heights_cm[i + 1]],
                    arrowprops=dict(arrowstyle="->", color="red"),
                    size=20,
                )
                ax.set_xlim(
                    -bay_widths[0] * 0.3, np.max(bay_widths_cm) + bay_widths[0] * 0.3
                )
        return fig, ax

    def portal_plotting(self):
        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
                bay_widths_cm = self.bay_abc_cm
                h_line = self.h_line_abc
                v_line = self.v_line_abc
            elif frame_face == "123":
                bay_widths = self.bay_123
                bay_widths_cm = self.bay_123_cm
                h_line = self.h_line_123
                v_line = self.v_line_123
            height = self.height
            heights_cm = self.heights_cm

            csf, cmt, bmt, bsd, axial_list = self.portal_computations()

            a = fig_f, ax_f = self.frame(f"PORTAL-COLUMN SHEAR DIAGRAM-{frame_face}",bay_widths,bay_widths_cm)
            a = fig_J, ax_J = self.frame(f"PORTAL-COLUMN AXIAL DIAGRAM-{frame_face}",bay_widths,bay_widths_cm)
            a = fig_g, ax_g = self.frame(f"PORTAL-COLUMN MOMENT DIAGRAM-{frame_face}",bay_widths,bay_widths_cm)
            a = fig_h, ax_h = self.frame(f"PORTAL-BEAM MOMENT DIAGRAM-{frame_face}",bay_widths,bay_widths_cm)
            a = fig_i, ax_i = self.frame(f"PORTAL-BEAM SHEAR DIAGRAM-{frame_face}",bay_widths,bay_widths_cm)
            for i in range(storey):
                for j in range(bay + 1):

                    ax_f.text(
                        bay_widths_cm[j],
                        heights_cm[i] + (height[i] / 2),
                        csf[::-1][i, j],fontsize = 12
                    )
                    rect = patches.Rectangle(
                        (bay_widths_cm[j], heights_cm[i]),
                        (csf[::-1][i, j] / np.max(csf)) * (np.mean(bay_widths_cm) / 4),
                        height[i],
                        linewidth=1,
                        edgecolor="r",
                        facecolor=self.color,
                        alpha=0.5,
                    )
                    ax_f.add_patch(rect)
                    axial_ = axial_list[i, j]
                    # Shear_diagram_beam
                    ax_J.text(
                        bay_widths_cm[j],
                        (heights_cm[i] + heights_cm[i + 1]) / 2,
                        axial_,
                    )
                    dy = -1
                    if axial_ > 0:
                        dy = 1

                    ax_J.arrow(
                        x=bay_widths_cm[j] + 0.8,
                        y=(heights_cm[i] + heights_cm[i + 1]) / 2,
                        dx=0,
                        dy=dy,
                        width=0.05,
                        ec="red",
                    )

                    # Moment_diagram_column
                    ax_g.text(
                        bay_widths_cm[j], heights_cm[i] + (height[i] / 2), cmt[i, j],fontsize = 12
                    )

                    ax_g.plot(
                        [
                            bay_widths_cm[j],
                            bay_widths_cm[j] + (cmt[i, j] * 0.01),
                            bay_widths_cm[j] - (cmt[i, j] * 0.01),
                            bay_widths_cm[j],
                        ],
                        [
                            heights_cm[i],
                            heights_cm[i],
                            heights_cm[i] + height[i],
                            heights_cm[i] + height[i],
                        ],
                        color="red",
                    )

                    if j != bay:
                        # Moment_diagram_beam
                        ax_h.text(
                            (bay_widths_cm[j] + bay_widths_cm[j + 1]) / 2,
                            heights_cm[i + 1],
                            bmt[i, j],fontsize = 12
                        )
                        ax_h.plot(
                            [
                                bay_widths_cm[j],
                                bay_widths_cm[j],
                                bay_widths_cm[j + 1],
                                bay_widths_cm[j + 1],
                            ],
                            [
                                heights_cm[i + 1],
                                heights_cm[i + 1]
                                + (
                                    (bmt[i, j] / np.max(bmt))
                                    * np.mean(bay_widths_cm)
                                    / 4
                                ),
                                heights_cm[i + 1]
                                - (
                                    (bmt[i, j] / np.max(bmt))
                                    * np.mean(bay_widths_cm)
                                    / 4
                                ),
                                heights_cm[i + 1],
                            ],
                            color="red",
                        )

                        sf = bsd[i, j]
                        # Shear_diagram_beam
                        ax_i.text(
                            (bay_widths_cm[j] + bay_widths_cm[j + 1]) / 2,
                            heights_cm[i + 1],
                            sf,fontsize = 12
                        )
                        rect = patches.Rectangle(
                            (bay_widths_cm[j], heights_cm[i + 1]),
                            bay_widths[j],
                            (csf[::-1][i, j] / np.max(csf)) * (np.mean(heights_cm) / 4),
                            linewidth=1,
                            edgecolor="r",
                            facecolor=self.color,
                            alpha=0.5,
                        )
                        ax_i.add_patch(rect)
            path = self.create_folder("Portal")
            a = fig_i.savefig(f"{path}\\Shear_diagram_beam-{frame_face}.png", dpi=500)
            a = fig_f.savefig(f"{path}\\Shear_force_column-{frame_face}.png", dpi=500)
            a = fig_J.savefig(f"{path}\\Axial_diagram_beam-{frame_face}.png", dpi=500)
            a = fig_g.savefig(f"{path}\\Moment_diagram_column-{frame_face}.png", dpi=500)
            a = fig_h.savefig(f"{path}\\Moment_diagram_beam-{frame_face}.png", dpi=500)

    ################################################################################################################################################################################################################################################################################################################
    # Portal Handcalcs
    jupyter = False
    params = "long"

    @handcalc(override=params, jupyter_display=jupyter)
    def column_shear_outer(self, shear=100):
        Base_shear = shear

        Column_Shear = Base_shear / 4

    @handcalc(override=params, jupyter_display=jupyter)
    def column_shear_inner(self, shear=100):
        Base_shear = shear

        Column_Shear = Base_shear / 2

    def column_sh(self, csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face):
        shear_unit1 = self.force_list[::-1].cumsum()
        shear_col_latex = ""
        shear_tr = ""
        for i in range(storey - 1, -1, -1):
            shear_th = ""
            for j in range(bay + 1):

                shear = shear_unit1[::-1][i]
                if j in [0, 2]:
                    latex, asd = self.column_shear_outer(shear)
                elif j == 1:
                    pos = "Inner"
                    latex, asd = self.column_shear_inner(shear)
                discription = (
                                f"Storey-{i+1}-Bay-{j+1}"
                            )
                name = (
                                f"col_shear_latex-{frame_face}"
                            )
                self.add_latex(
                                key=name, values=latex, discription=discription
                            )

        return shear_col_latex

    @handcalc(override=params, jupyter_display=jupyter)
    def column_moment_computer(self, shear, column_height):
        column_shear = shear
        column_height = column_height
        column_moment = shear * column_height / 2

    def column_mom(self, csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face):
        moment_col_latex = ""
        moment_col_tr = ""
        for i in range(storey - 1, -1, -1):
            moment_th = ""
            for j in range(bay + 1):
                shear = csf[::-1][i, j]
                column_height = self.height[i]
                latex, asd = self.column_moment_computer(shear, column_height)
                discription = (
                                f"Storey-{i+1}-Bay-{j+1}"
                            )
                name = (
                                f"col_moment_latex-{frame_face}"
                            )
                self.add_latex(
                                key=name, values=latex, discription=discription
                            )

            
        return moment_col_latex

    @handcalc(override=params, jupyter_display=jupyter)
    def beam_moment_computer1(self, col_moment1, col_moment2):
        beam_moment = col_moment1 + col_moment2

    @handcalc(override=params, jupyter_display=jupyter)
    def beam_moment_computer2(self, col_moment1):
        beam_moment = col_moment1

    def beam_mom(self, csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face):
        moment_beam_latex = ""
        moment_beam_tr = ""
        for i in range(storey - 1, -1, -1):
            moment_th = ""
            for j in range(bay):
                if i + 1 <= storey - 1:
                    column_moment1 = cmt[i, 0]
                    column_moment2 = cmt[i + 1, 0]
                    latex, asd = self.beam_moment_computer1(
                        column_moment1, column_moment2
                    )
                else:
                    column_moment1 = cmt[i, 0]
                    latex, asd = self.beam_moment_computer2(column_moment1)
                discription = (
                                f"Storey-{i+1}-Bay-{j+1}"
                            )
                name = (
                                f"beam_moment_latex-{frame_face}"
                            )
                self.add_latex(
                                key=name, values=latex, discription=discription
                            )


        return moment_beam_latex

    @handcalc(override=params, jupyter_display=jupyter)
    def beam_shear_computer(self, beam_moment, bay_width):
        beam_moment = beam_moment
        bay_width = bay_width
        Shear_beam = (beam_moment * 2) / bay_width

    def beam_sh(self, csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face):
        shear_beam_latex = ""
        shear_beam_tr = ""
        for i in range(storey - 1, -1, -1):
            shear_th = ""
            for j in range(bay + 1):
                if j != bay:
                    beam_moment = bmt[i, j]
                    sf = np.round(bmt[i, j] / (bay_widths[j] * 0.5), 2)
                    bay_width = bay_widths[j]
                    latex, asd = self.beam_shear_computer(beam_moment, bay_width)
                    discription = (
                                f"Storey-{i+1}-Bay-{j+1}"
                            )
                    name = (
                                f"beam_shear_latex-{frame_face}"
                            )
                    self.add_latex(
                                key=name, values=latex, discription=discription
                            )

        return shear_beam_latex

    def portal_latex_compiler(
        self, csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face
    ):
        col_shear_latex = self.column_sh(
            csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face
        )
        col_moment_latex = self.column_mom(
            csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face
        )
        beam_moment_latex = self.beam_mom(
            csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face
        )
        beam_shear_latex = self.beam_sh(
            csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face
        )
        return col_shear_latex + col_moment_latex + beam_moment_latex + beam_shear_latex

    def handcalc_portal(self):
        self.portal_computations()
        path = self.path_data
        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
                bay_widths_cm = self.bay_abc_cm
            else:
                bay_widths = self.bay_123
                bay_widths_cm = self.bay_123_cm

            csf = (
                pd.read_csv(
                    f"{path}\\csf-{frame_face}.csv"
                )
            ).to_numpy()
            print(csf)
            cmt = (
                pd.read_csv(
                    f"{path}\\cmt-{frame_face}.csv"
                )
            ).to_numpy()  # Bending moment for column
            bmt = (
                pd.read_csv(
                    f"{path}\\bmt-{frame_face}.csv"
                )
            ).to_numpy()  # Bending moment for beam
            latex = self.portal_latex_compiler(
                csf, cmt, bmt, bay_widths, bay_widths_cm, frame_face
            )

    def selenium_portal(self):
        driver = webdriver.Chrome(r"C:\Users\63976\Desktop\chromedriver.exe")
        for frame_face_ in self.frame_face:
            id_list = [
                f"col_shear_latex-{frame_face_}",
                f"col_moment_latex-{frame_face_}",
                f"beam_moment_latex-{frame_face_}",
                f"beam_shear_latex-{frame_face_}",
            ]
            driver.get(
                f"C:\\Users\\63976\\Documents\\Xlwings\\pihuts\\hl\\Handcalc_Portal_Html\\portal--{frame_face_}.html"
            )
            path = self.create_folder("Portal_Pictures")
            driver.maximize_window()

            for x in id_list:
                element = driver.find_element(By.ID, x)
                driver.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(0.5)
                element.screenshot(f"{path}\\{x}.png")
        driver.quit()

    ####################################################################################################################################
    ####Frame without Point load/ FrameABC
    def frame_distributed(self, title,bay_widths_cm):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(1, 1, 1)
        storey = self.storey
        bay = self.bay
        heights_cm = self.heights_cm
        # bay_widths_cm = self.bay_abc_cm
        plt.title(label=title, loc="left", fontsize=16, color="green")
        for i in range(storey):
            for j in range(bay):
                ax.plot(
                    [bay_widths_cm[j], bay_widths_cm[j + 1]],
                    [heights_cm[i + 1], heights_cm[i + 1]],
                    color="blue",
                )  # Horizontal line

        for i in range(storey):
            for j in range(bay + 1):
                ax.plot(
                    [bay_widths_cm[j], bay_widths_cm[j]],
                    [heights_cm[i], heights_cm[i + 1]],
                    color="blue",
                )
        return fig, ax

    def distributed_load_calculation(self):
        beam_w = self.beam_w
        slab_w = self.slab_w
        wall_w = self.wall_w
        parapits_w = self.parapits_w
        storey = self.storey
        struc = self.no_frames
        bay = self.bay
        live_load = self.live_load
        bay_widths = self.bay_abc
        slab_force_dead = np.zeros((struc, storey, bay))
        slab_force_live = np.zeros((struc, storey, bay))
        force_list_dead = np.ones((struc, storey, bay))
        force_list_live = np.ones((struc, storey, bay))
        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
            else:
                bay_widths = self.bay_123
            for load_type in self.load_types:
                for frame_ in np.arange(struc):
                    for i in range(storey):
                        for j in range(bay):
                            if frame_face == "abc":
                                if load_type == "Dead":

                                    if frame_ == 0:
                                        slab_force_dead[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_])
                                                * beam_w
                                            )
                                            / 4,
                                            2,
                                        )
                                    elif frame_ == 1:
                                        slab_force_dead[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_])
                                                * beam_w
                                            )
                                            / 4
                                            + (
                                                bay_widths[j]
                                                * (bay_widths[frame_ - 1])
                                                * beam_w
                                            )
                                            / 4,
                                            2,
                                        )
                                    else:
                                        slab_force_dead[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_ - 1])
                                                * beam_w
                                            )
                                            / 4,
                                            2,
                                        )

                                    # force_list[i,j] = 1.5
                                    if frame_ != 1:
                                        if i != storey - 1:
                                            wu = (
                                                beam_w
                                                + wall_w
                                                + slab_force_dead[frame_, i, j]
                                            )

                                        else:
                                            wu = (
                                                beam_w
                                                + parapits_w
                                                + slab_force_dead[frame_, i, j]
                                            )
                                    else:
                                        wu = beam_w + slab_force_dead[frame_, i, j]

                                    wu = np.round(wu, 2)
                                    force_list_dead[frame_, i, j] = wu
                                else:
                                    if frame_ == 0:
                                        slab_force_live[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_])
                                                * live_load
                                            )
                                            / 2,
                                            2,
                                        )
                                    elif frame_ == 1:
                                        slab_force_live[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_])
                                                * live_load
                                            )
                                            / 2
                                            + (
                                                bay_widths[j]
                                                * (bay_widths[frame_ - 1])
                                                * live_load
                                            )
                                            / 2,
                                            2,
                                        )
                                    else:
                                        slab_force_live[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_ - 1])
                                                * live_load
                                            )
                                            / 2,
                                            2,
                                        )

                                    wu = np.round(wu, 2)
                                    wu = slab_force_live[frame_, i, j]
                                    force_list_live[frame_, i, j] = wu

                            elif frame_face == "123":
                                if load_type == "Dead":
                                    if frame_ != 1:  # Frame load
                                        if i != storey - 1:
                                            wu = beam_w + wall_w
                                        else:
                                            wu = beam_w + parapits_w
                                    else:
                                        wu = beam_w
                                    wu = np.round(wu, 2)
                                    force_list_dead[frame_, i, j] = wu

                                    if frame_ == 0:
                                        slab_force_dead[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_])
                                                * beam_w
                                            )
                                            / 4,
                                            2,
                                        )
                                    elif frame_ == 1:
                                        slab_force_dead[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_])
                                                * beam_w
                                            )
                                            / 4
                                            + (
                                                bay_widths[j]
                                                * (bay_widths[frame_ - 1])
                                                * beam_w
                                            )
                                            / 4,
                                            2,
                                        )
                                    else:
                                        slab_force_dead[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_ - 1])
                                                * beam_w
                                            )
                                            / 4,
                                            2,
                                        )
                                else:
                                    force_list_live[frame_, i, j] = 0
                                    if frame_ == 0:
                                        slab_force_live[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_])
                                                * live_load
                                            )
                                            / 4,
                                            2,
                                        )
                                    elif frame_ == 1:
                                        slab_force_live[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_])
                                                * live_load
                                            )
                                            / 4
                                            + (
                                                bay_widths[j]
                                                * (bay_widths[frame_ - 1])
                                                * live_load
                                            )
                                            / 4,
                                            2,
                                        )
                                    else:
                                        slab_force_live[frame_, i, j] = np.round(
                                            (
                                                bay_widths[j]
                                                * (bay_widths[frame_ - 1])
                                                * live_load
                                            )
                                            / 4,
                                            2,
                                        )
                path = self.path_data
                np.save(f"{path}\\dead_distributed-{frame_face}.npy", force_list_dead)
                np.save(f"{path}\\live_distributed-{frame_face}.npy", force_list_live)
                np.save(f"{path}\\dead_slab-force-{frame_face}.npy", slab_force_dead)
                np.save(f"{path}\\live_slab-force-{frame_face}.npy", slab_force_live)

    def Moment_line(self, w, l):
        moment = (w * (0.8 * l) ** 2) / 8
        sf_ss_end = (w * (0.8 * l)) / 2
        end_moment = (sf_ss_end * (0.1 * l)) + (w * 0.1 * l * (0.1 * l * 0.5))
        x = np.linspace(-l / 2, l / 2, 30)
        y = (((-end_moment - moment) / (-l / 2) ** 2) * (x**2)) + moment

        return x + l / 2, y, moment, end_moment, sf_ss_end, sf_ss_end + (w * 0.1 * l)

    def Moment_line1(self, w, l,p=0):
        moment = (w * (0.8 * l) ** 2) / 8 + (p * l) / 4
        sf_ss_end = (w * (0.8 * l)) / 2 + p / 2
        end_moment = (sf_ss_end * (0.1 * l)) + (w * 0.1 * l * (0.1 * l * 0.5))
        x = np.linspace(-l / 2, l / 2, 30)
        y = (((-end_moment - moment) / (-l / 2) ** 2) * (x**2)) + moment
        return x + l / 2, y, moment, end_moment, sf_ss_end, sf_ss_end + (w * 0.1 * l)
    # Bending moment diagram for beams
    def distributed_frame_display(self):
        storey = self.storey
        struc = self.no_frames
        bay = self.bay
        path_data = self.path_data
        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
                bay_widths_cm = self.bay_abc_cm
            else:
                bay_widths = self.bay_123
                bay_widths_cm = self.bay_123_cm

            for load_type in self.load_types:
                if load_type == "Dead":
                    force_list = np.load(f"{path_data}\\dead_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path_data}\\dead_slab-force-{frame_face}.npy")
                else:
                    force_list = np.load(f"{path_data}\\live_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path_data}\\live_slab-force-{frame_face}.npy")
                # force_list = np.load(f"dead_distributed-{frame_face}.npy")
                for frame_ in np.arange(struc):
                    if frame_face == "abc":
                        name = f"APPROXIMATE METHOD-FRAME-{chr(65+frame_)}-{load_type}"
                    else:
                        name = f"APPROXIMATE METHOD-FRAME-{frame_+1}-{load_type}"
                    a = fig, ax = self.frame_distributed(name,bay_widths_cm)
                    arrow_number = 15

                    for i in range(storey):
                        arl = 5
                        for j in range(bay):
                            if (force_list[frame_,i,j]) < 1:
                                distributed_load_height = 0
                            else:

                                distributed_load_height = (((force_list[frame_,i,j])/np.max(force_list)) * (np.mean(height)/3))
                            # a = ax.text((bay_widths_cm[j]+bay_widths_cm[j+1])/2,heights_cm[i+1] + distributed_load_height , force_list[frame_,i,j])
                            if frame_face == "123":
                                distributed_load_height1 = 0.5
                                distributed_load_height2 = distributed_load_height
                                a =ax.text(
                                    (bay_widths_cm[j] + bay_widths_cm[j + 1]) / 2,
                                    heights_cm[i + 1]
                                    + distributed_load_height1
                                    + distributed_load_height2,
                                    slab_force[frame_, i, j],fontsize = 12
                                )
                                a =ax.arrow(
                                    x=(bay_widths_cm[j] + bay_widths_cm[j + 1]) / 2,
                                    y=heights_cm[i + 1]
                                    + distributed_load_height1
                                    + distributed_load_height2
                                    + 0.3,
                                    dx=0,
                                    dy=-distributed_load_height1,
                                    width=0.05,
                                    ec="red",
                                    head_starts_at_zero=False,
                                )

                            a =ax.text(
                                (bay_widths_cm[j] + bay_widths_cm[j + 1]) / 2,
                                heights_cm[i + 1] + distributed_load_height,
                                force_list[frame_, i, j],fontsize = 12
                            )
                            for k in range(arrow_number):
                                a =ax.annotate(
                                    "",
                                    xytext=[
                                        bay_widths_cm[j]
                                        + (bay_widths[j] / arrow_number) * k,
                                        heights_cm[i + 1] + distributed_load_height,
                                    ],
                                    xy=[
                                        bay_widths_cm[j]
                                        + (bay_widths[j] / arrow_number) * k,
                                        heights_cm[i + 1],
                                    ],
                                    arrowprops=dict(arrowstyle="->", color="red"),
                                    fontsize=12,
                                )
                            rect = patches.Rectangle(
                                (bay_widths_cm[j], heights_cm[i + 1]),
                                bay_widths[j],
                                distributed_load_height,
                                linewidth=1,
                                edgecolor="r",
                                facecolor=self.color,
                                alpha=0.5,
                            )
                            a =ax.add_patch(rect)
                    # ax.set_xlim(-bay_widths[0]*0.3, np.max(bay_widths_cm)+bay_widths[0]*0.3)
                    # ax.set_ylim(-height*0.5, (height*storey)+(height*0.5))
                    path = self.create_folder("Approximate_method_abc")
                    path1 = self.create_folder("Data")
                    
                    a = fig.savefig(f"{path}\\{name}.png", dpi=300)
                    

    def beam_bending_moment_distributed(self):
        beam_w = self.beam_w
        slab_w = self.slab_w
        wall_w = self.wall_w
        parapits_w = self.parapits_w
        storey = self.storey
        struc = self.no_frames
        bay = self.bay
        live_load = self.live_load
        bay_widths = self.bay_abc
        columns_bending = np.zeros(
            (struc, storey, bay + 1)
        )  # Bending moment for column
        columns_shear = np.zeros((struc, storey, bay + 1))  # Bending moment for column
        beams_bending = np.zeros((struc, storey, bay))  # Bending moment for beam
        beams_mid_bending = np.zeros((struc, storey, bay))  # Bending moment for beam
        beams_shear = np.zeros((struc, storey, bay))  # Bending moment for beam
        end_bm = np.ones((storey, bay))
        path_data = self.path_data
        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
                bay_widths_cm = self.bay_abc_cm
            else:
                bay_widths = self.bay_123
                bay_widths_cm = self.bay_123_cm

            for load_type in self.load_types:
                if load_type == "Dead":

                    force_list = np.load(f"{path_data}\\dead_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path_data}\\dead_slab-force-{frame_face}.npy")
                else:
                    force_list = np.load(f"{path_data}\\live_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path_data}\\live_slab-force-{frame_face}.npy")
                for frame_ in np.arange(struc):
                    if frame_face == "abc":
                        name = f"APPROXIMATE-BEAM MOMENT DIAGRAM-FRAME-{chr(65+frame_)}-{load_type}"
                    else:
                        name = f"APPROXIMATE-BEAM MOMENT DIAGRAM-FRAME-{frame_+1}-{load_type}"
                    a = fig_bm, ax_bm = self.frame_distributed(name,bay_widths_cm)
                    for i in range(storey):
                        for j in range(bay):
                            if frame_face == "abc":
                                beam_output = self.Moment_line(
                                    force_list[frame_, i, j], bay_widths[j]
                                )
                            else:
                                beam_output = self.Moment_line1(
                                    force_list[frame_, i, j],
                                    bay_widths[j],
                                    slab_force[frame_, i, j],
                                )
                            x = beam_output[0]
                            y = beam_output[1]
                            mid_moment = np.round(beam_output[2], 2)
                            beams_mid_bending[frame_,i, j] = mid_moment
                            end_moment = np.round(beam_output[3], 2)
                            end_bm[ i, j] = end_moment
                            beams_bending[frame_,i, j]  = end_moment
                            ax_bm.fill_between(
                                bay_widths_cm[j] + x,
                                heights_cm[i + 1] + (y * 0.05),
                                heights_cm[i + 1],
                                color=self.color,
                            )
                            ax_bm.text(
                                (bay_widths_cm[j] + bay_widths_cm[j + 1]) / 2,
                                heights_cm[i + 1],
                                mid_moment,fontsize = 12
                            )
                            ax_bm.text(
                                bay_widths_cm[j],
                                heights_cm[i + 1] - (height[i] * 0.35),
                                end_moment,fontsize = 12
                            )
                            ax_bm.text(
                                bay_widths_cm[j + 1] - bay_widths[j] * 0.15,
                                heights_cm[i + 1] - (height[i] * 0.35),
                                end_moment,fontsize = 12
                            )

                    ax_bm.set_xlim(
                        -bay_widths[0] * 0.3,
                        np.max(bay_widths_cm) + bay_widths[0] * 0.3,
                    )
                    path = self.path_approximate
                    path1 = self.path_data

                    a = fig_bm.savefig(
                        f"{path}\\{name}.png",
                        dpi=300,
                    )

                np.save(
                        f"{path1}\\beams_bending1_{load_type}_{frame_face}.npy",
                        beams_bending,
                    )
                np.save(
                        f"{path1}\\beams__mid_bending1_{load_type}_{frame_face}.npy",
                        beams_mid_bending,
                    )
                np.save(
                        f"{path1}\\end_bm_{load_type}_{frame_face}.npy", end_bm
                    )
                # Shear force diagram for beams

    def beam_shear_distributed(self):
        beam_w = self.beam_w
        slab_w = self.slab_w
        wall_w = self.wall_w
        parapits_w = self.parapits_w
        storey = self.storey
        struc = self.no_frames
        bay = self.bay
        live_load = self.live_load
        bay_widths = self.bay_abc
        columns_bending = np.zeros(
            (struc, storey, bay + 1)
        )  # Bending moment for column
        columns_shear = np.zeros((struc, storey, bay + 1))  # Bending moment for column
        beams_bending = np.zeros((struc, storey, bay))  # Bending moment for beam
        beams_mid_bending = np.zeros((struc, storey, bay))  # Bending moment for beam
        beams_shear = np.zeros((struc, storey, bay))  # Bending moment for beam
        path_data = self.path_data
        end_bm = np.ones((storey, bay))

        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
            else:
                bay_widths = self.bay_123

            for load_type in self.load_types:
                if load_type == "Dead":
                    force_list = np.load(f"{path_data}\\dead_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path_data}\\dead_slab-force-{frame_face}.npy")
                else:
                    force_list = np.load(f"{path_data}\\live_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path_data}\\live_slab-force-{frame_face}.npy")
                for frame_ in np.arange(struc):
                    if frame_face == "abc":
                        name = f"APPROXIMATE-BEAM SHEAR DIAGRAM-FRAME-{chr(65+frame_)}-{load_type}"
                    else:
                        name = f"APPROXIMATE-BEAM SHEAR DIAGRAM-FRAME-{frame_+1}-{load_type}"
                    a = fig_bs, ax_bs = self.frame_distributed(name,bay_widths_cm)
                    for i in range(storey):
                        for j in range(bay):
                            if frame_face == "abc":
                                beam_output = self.Moment_line(
                                    force_list[frame_, i, j], bay_widths[j]
                                )
                            else:
                                beam_output = self.Moment_line1(
                                    force_list[frame_, i, j],
                                    bay_widths[j],
                                    slab_force[frame_, i, j],
                                )
                            end_shear = np.round(beam_output[5], 2)
                            beams_shear[frame_,i, j]  = end_shear
                            ax_bs.fill_between(
                                [
                                    bay_widths_cm[j],
                                    bay_widths_cm[j],
                                    bay_widths_cm[j + 1],
                                    bay_widths_cm[j + 1],
                                ],
                                [
                                    heights_cm[i + 1],
                                    heights_cm[i + 1] + end_shear * 0.05,
                                    heights_cm[i + 1] - end_shear * 0.05,
                                    heights_cm[i + 1],
                                ],
                                heights_cm[i + 1],
                                color=self.color,
                            )
                            ax_bs.text(
                                (bay_widths_cm[j] + bay_widths_cm[j + 1]) / 2,
                                heights_cm[i + 1],
                                end_shear,fontsize = 12
                            )

                            # ax_bm.text(bay_widths_cm[j], heights_cm[i+1]-(height*0.35) , end_moment)
                            # ax_bm.text(bay_widths_cm[j+1]-bay_widths[j]*0.15, heights_cm[i+1]-(height*0.35) , end_moment)
                    path = self.path_approximate
                    path1 = self.path_data

                    ax_bs.set_xlim(
                        -bay_widths[0] * 0.3,
                        np.max(bay_widths_cm) + bay_widths[0] * 0.3,
                    )

                    a = fig_bs.savefig(
                        f"{path}\\{name}.png",
                        dpi=300,
                    )

                np.save(
                        f"{path1}\\beams_shear1_{load_type}_{frame_face}.npy",
                        beams_shear,
                    )
                # Bending moment diagram for columns

    def col_moment_distribution(self):
        beam_w = self.beam_w
        slab_w = self.slab_w
        wall_w = self.wall_w
        parapits_w = self.parapits_w
        storey = self.storey
        struc = self.no_frames
        bay = self.bay
        live_load = self.live_load
        bay_widths = self.bay_abc
        columns_bending = np.zeros(
            (struc, storey, bay + 1)
        )  # Bending moment for column
        columns_shear = np.zeros((struc, storey, bay + 1))  # Bending moment for column
        beams_bending = np.zeros((struc, storey, bay))  # Bending moment for 
        beams_mid_bending = np.zeros((struc, storey, bay))  # Bending moment for beam
        beams_shear = np.zeros((struc, storey, bay))  # Bending moment for beam
        path_data = self.path_data

        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
            else:
                bay_widths = self.bay_123

            for load_type in self.load_types:
                if load_type == "Dead":
                    force_list = np.load(f"{path_data}\\dead_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path_data}\\dead_slab-force-{frame_face}.npy")
                else:
                    force_list = np.load(f"{path_data}\\live_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path_data}\\live_slab-force-{frame_face}.npy")
                for frame_ in np.arange(struc):
                    end_bm = np.load(
                        f"{path_data}\\end_bm_{load_type}_{frame_face}.npy"
                    )
                    end_cm = np.ones((storey, bay+1))
                    end_cm[:,0] = end_bm[:,0][::-1].cumsum()
                    end_cm[:,-1] = end_bm[:,-1][::-1].cumsum()
                    end_cm[:,1:-1] = np.reshape(np.diff(end_bm)[::-1].cumsum(), np.shape(end_cm[:,1:-1]))
                    if frame_face == "abc":
                        name = f"APPROXIMATE-COLOUMN MOMENT DIAGRAM-FRAME-{chr(65+frame_)}-{load_type}"
                    else:
                        name = f"APPROXIMATE-COLOUMN MOMENT DIAGRAM-FRAME-{frame_+1}-{load_type}"
                    a = fig_cm, ax_cm = self.frame_distributed(name,bay_widths_cm)
                    for i in range(storey):
                        for j in range(bay + 1):
                            moment = np.round(end_cm[::-1][i, j], 2)
                            columns_bending[i, j] = moment
                            ax_cm.text(
                                bay_widths_cm[j],
                                heights_cm[i] + (height[i] / 2),
                                moment,fontsize = 12
                            )
                            scale = 0.03
                            ax_cm.fill_betweenx(
                                [
                                    heights_cm[i],
                                    heights_cm[i],
                                    heights_cm[i] + height[i],
                                    heights_cm[i] + height[i],
                                ],
                                [
                                    bay_widths_cm[j],
                                    bay_widths_cm[j] + (moment * scale),
                                    bay_widths_cm[j] - (moment * scale),
                                    bay_widths_cm[j],
                                ],
                                bay_widths_cm[j],
                                color=self.color,
                                interpolate=True,
                            )
                        # ax_f.add_patch(rect)

                # ax_cm.set_xlim(-bay_widths[0]*0.5, np.max(bay_widths_cm)+bay_widths[0]*0.5)
                path = self.path_approximate
                path1 = self.path_data
                a = fig_cm.savefig(
                    f"{path}\{name}.png",
                    dpi=300,
                )
                np.save(f"{path1}\\col_moment1_{load_type}_{frame_}_{frame_face}.npy", end_cm)

    def approxiate_method_figures(self):
        self.distributed_load_calculation()
        self.distributed_frame_display
        self.beam_bending_moment_distributed()
        self.beam_shear_distributed()
        self.col_moment_distribution()

    def html_makerl(self):
        shear_beam_latex = ""
        data = self.data
        for keys in data.keys():
            shear_tr = ""
            for floor in range(self.storey):
                shear_th = ""
                scale = int(ceil((len(data[keys])) / (self.storey))) 
                if scale == 1:
                    scale = 3
                for divider in range(scale):
                    try:
                        latex_data = data[keys][(floor * scale) + divider]
                        latex = latex_data[0]
                        discription = latex_data[1]
                        shear_th += f"<td><h4>{discription}</h4><p style='font-size:10px;'>{latex}</p></td>"
                        # self.td[keys].append([values,discription])
                    except:
                        pass
                shear_tr += f"<tr>{shear_th}</tr>"
            shear_beam_latex += f""" <div class="container">
                        <div class="table-div"> <table  id= "{keys}" >{shear_tr}</table></div>
                        </div>"""

        style = r"{border: 1px solid;}"
        style_container = """{
                    position:absolute;
                    right:0;
                    top:0;
                    }"""
        style_container1 = """{
                    position:relative; 
                    padding-right:'images-width';
                    }"""
        style_body = """{
font-family: Garamond, serif;
}"""
        div = f"""<!DOCTYPE html>
                    <html>

                    <head>
                        <meta charset="utf-8">
                        <meta name="viewport" content="width=device-width">
                        <title>MathJax example</title>
                        <script src="https://polyfill.io/v3/polyfill.min.js?features=es6"></script>
                        <script id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
                        </script>
                    <style>
                    body {
  style_body
}
                    table, th, td {style}

                    .container img {style_container}
                    .container .table-div  {style_container1}
                    </style>
                    </head>

                    <body>
                {shear_beam_latex}
                    </body>

                    </html>"""
        path = self.create_folder("Data")
        with open(f"{path}\\All_Calculations.html", "w",encoding="utf-8") as file:
            file.write(div)
    def html_makerl1(self):
            shear_beam_latex = ""
            data = self.data1
            for keys in data.keys():
                shear_tr = ""
                for floor in range(self.storey):
                    shear_th = ""
                    scale = int(ceil((len(data[keys])) / (self.storey))) 
                    if scale == 1:
                        scale = 3
                    for divider in range(scale):
                        try:
                            latex_data = data[keys][(floor * scale) + divider]
                            latex = latex_data[0]
                            discription = latex_data[1]
                            shear_th += f"<td><h4>{discription}</h4><p style='font-size:10px;'>{latex}</p></td>"
                            # self.td[keys].append([values,discription])
                        except:
                            pass
                    shear_tr += f"<tr>{shear_th}</tr>"
                shear_beam_latex += f""" <div class="container">
                            <div class="table-div"> <table  id= "{keys}" >{shear_tr}</table></div>
                            </div>"""

            style = r"{border: 1px solid;}"
            style_container = """{
                        position:absolute;
                        right:0;
                        top:0;
                        }"""
            style_container1 = """{
                        position:relative; 
                        padding-right:'images-width';
                        }"""
            style_body = """{
    font-family: Garamond, serif;
    }"""
            div = f"""<!DOCTYPE html>
                        <html>

                        <head>
                            <meta charset="utf-8">
                            <meta name="viewport" content="width=device-width">
                            <title>MathJax example</title>
                            <script src="https://polyfill.io/v3/polyfill.min.js?features=es6"></script>
                            <script id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
                            </script>
                        <style>
                        body {
    style_body
    }
                        table, th, td {style}

                        .container img {style_container}
                        .container .table-div  {style_container1}
                        </style>
                        </head>

                        <body>
                    {shear_beam_latex}
                        </body>

                        </html>"""
            path = self.create_folder("Data")
            with open(f"{path}\\All_Calculations1.html", "w",encoding="utf-8") as file:
                file.write(div)
    def html_makerl_flex(self):
        shear_beam_latex = ""
        data = self.data
        for keys in data.keys():
            shear_th = ""
            for latex_order,content in enumerate(data[keys]):
                latex_data = data[keys][latex_order]
                latex = latex_data[0]
                discription = latex_data[1]
                if len(data[keys]) > 8:
                    size = 5
                else:size = 10
                shear_th += f"<div><p class= 'big' font-size: {size}px; ><strong>{discription}</strong><br>{latex}</p></div>"

            shear_beam_latex += f""" <div class="flex-container" id = "{keys}" >{shear_th}</div>
                       """
            # for content in data[keys]:
            # shear_tr = ""


        style = r"{border: 1px solid;}"
        style_container = """{
  display: inline-flex;
  flex-wrap: wrap;
  background-color: DodgerBlue;
  align-content: stretch;
  max-width: 1000px;
}"""
        style_container1 = """{
  background-color: #f1f1f1;
  margin: 10px;
  text-align: center;
  line-height: 75px;
  font-size: 10px
  
}"""
        style_body = """{
font-family: Garamond, serif;
}"""
        body = """{
        height: 842px;
        width: 595px;
        /* to centre page on screen*/
        margin-left: auto;
        margin-right: auto;
    }"""
        p_style =""" {
  line-height: 0.7;
}"""
        div = f'''<!DOCTYPE html>
<html>
<head>
                        <meta charset="utf-8">
                        <meta name="viewport" content="width=device-width">
                        <title>MathJax example</title>
                        <script src="https://polyfill.io/v3/polyfill.min.js?features=es6"></script>
                        <script id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
                        </script>
<style>

.flex-container {
  style_container
}
.flex-container > div {
  style_container1
}
p.big {
  p_style
}
</style>
</head>
<body>

<h1>The flex-wrap Property</h1>

<p>The "flex-wrap: wrap;" specifies that the flex items will wrap if necessary:</p>

{shear_beam_latex
}
<p>Try resizing the browser window.</p>

</body>
</html>'''
        path = self.create_folder("Data")
        with open(f"{path}\\All_Calculations_new.html", "w") as file:
            file.write(div)
    def html_makerl_flex_beta(self):
        shear_beam_latex = ""
        data = self.data
        for keys in data.keys():
            shear_th = ""
            for latex_order,content in enumerate(data[keys]):
                latex_data = data[keys][latex_order]
                latex = latex_data[0]
                discription = latex_data[1]
                if len(data[keys]) > 8:
                    size = 5
                else:size = 10
                shear_th += f"<div>{latex}</div>"

            shear_beam_latex += f""" <div class="grid" id = "{keys}" >{shear_th}</div>
                       """
            # for content in data[keys]:
            # shear_tr = ""


        style = r"{border: 1px solid;}"
        style_container = """{
  display: inline-grid;
  grid-auto-rows: 2fr 2fr 1fr;
  grid-auto-columns: 1fr;
}"""
        grid1 = """
        {
  grid-column-start: span 2;
}
        """
        grid2 ="""{
  grid-column-start: 2;
}"""
        grid3 ="""{
  aspect-ratio: 1;
  grid-gap: 5px;
  outline: 2px solid red;
  vertical-align: top;
  margin: 10px;
  counter-reset: num;
}"""
        grid4 ="""{
  border: 2px solid;
  font-size: 10px;
  box-sizing: border-box;
  font-family: sans-serif;
  display: grid;
  place-content: center;
}"""
        grid5 =""" {
  content: counter(num);
  counter-increment: num;
}"""


        style_container1 = """{
  background-color: #f1f1f1;
  margin: 10px;
  text-align: center;
  line-height: 75px;
  
}"""
        style_body = """{
font-family: Garamond, serif;
}"""
        div = f'''<!DOCTYPE html>
<html>
<head>
                        <meta charset="utf-8">
                        <meta name="viewport" content="width=device-width">
                        <title>MathJax example</title>
                        <script src="https://polyfill.io/v3/polyfill.min.js?features=es6"></script>
                        <script id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
                        </script>
<style>
.grid {
  style_container
}
.grid :nth-child(1):nth-last-child(odd) {
  grid1
}
.grid :nth-child(4):nth-last-child(odd) {
  grid2
}

/**/
.grid {
  grid3
}
.grid *{
  grid4
}
.grid *:before {
  grid5
}

</style>
</head>
<body>



{shear_beam_latex}


</body>
</html>'''
        path = self.create_folder("Data")
        with open(f"{path}\\All_Calculations_new.html", "w") as file:
            file.write(div)
    def add_latex(self, key, values, discription):

        # if type(values) == list:
        #     print("is a list")
        #     for value in values:
        #         self.data[key].append(value)
        # else:
        self.data[key].append([values, discription])
    def add_latex1(self, key, values, discription):

        # if type(values) == list:
        #     print("is a list")
        #     for value in values:
        #         self.data[key].append(value)
        # else:
        self.data1[key].append([values, discription])

    @handcalc(override=params, jupyter_display=jupyter)
    def beam_moment(self, w, l):
        moment = (w * (0.8 * l) ** 2) / 8
        shear_end = (w * (0.8 * l)) / 2
        end_moment = (shear_end * (0.1 * l)) + (w * ((0.1 * l) ** 2) / 2)
    @handcalc(override=params, jupyter_display=jupyter)
    def beam_moment1(self,w, l, p):
        moment = (w * (0.8 * l) ** 2) / 8 + (p * l) / 4
        shear_end = (w * (0.8 * l)) / 2 + p / 2
        end_moment = (shear_end * (0.1 * l)) + (w * ((0.1 * l) ** 2) / 2)

    @handcalc(override=params, jupyter_display=jupyter)
    def beam_shear(self, w, l):
        shear = (w * (0.8 * l)) / 2
        shear_end = shear + (w * 0.1 * l)

    @handcalc(override=params, jupyter_display=jupyter)
    def beam_shear1(self,w, l, p):
        shear = (w * (0.8 * l)) / 2 + p / 2
        shear_end = shear + (w * 0.1 * l)

    def beam_sh_approximate(self):
        shear_beam_latex = ""
        struc = self.no_frames
        shear_tr = ""
        path = self.path_data
        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
            else:
                bay_widths = self.bay_123

            for load_type in self.load_types:
                if load_type == "Dead":
                    force_list = np.load(f"{path}\\dead_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path}\\dead_slab-force-{frame_face}.npy")
                else:
                    force_list = np.load(f"{path}\\live_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path}\\live_slab-force-{frame_face}.npy")
                for frame_ in np.arange(2):
                    for i in range(storey - 1, 0, -1):
                        shear_th = ""
                        for j in range(bay):
                            if frame_face == "abc":
                                latex, asd = self.beam_shear(
                                    force_list[frame_, i, j], bay_widths[j]
                                )
                                if frame_ == 0:
                                    frame = [chr(65), chr(65 + 2)]
                                else:
                                    frame = chr(65 + frame_)
                            else:
                                latex, asd = self.beam_shear1(
                                    force_list[frame_, i, j],
                                    bay_widths[j],
                                    slab_force[frame_, i, j],
                                )
                                if frame_ == 0:
                                    frame = [frame_ + 1, frame_ + 3]
                                else:
                                    frame = frame_ + 2
                            if i == 1:
                                i_ = [i + 1, i]
                            else:
                                i_ = i + 1

                            discription = (
                                f"Storey-{i_}-Bay-{j+1}-Frame-{frame}-Load-{load_type}"
                            )
                            name = f"beam_shear_latex-{load_type}_{frame}_{frame_face}"
                            self.add_latex(
                                key=name, values=latex, discription=discription
                            )

    def beam_mom_approximate(self):
        shear_col_latex = ""
        struc = self.no_frames
        shear_tr = ""
        path = self.path_data
        for frame_face in self.frame_face:
            if frame_face == "abc":
                bay_widths = self.bay_abc
            else:
                bay_widths = self.bay_123

            for load_type in self.load_types:
                if load_type == "Dead":
                    force_list = np.load(f"{path}\\dead_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path}\\dead_slab-force-{frame_face}.npy")
                else:
                    force_list = np.load(f"{path}\\live_distributed-{frame_face}.npy")
                    slab_force = np.load(f"{path}\\live_slab-force-{frame_face}.npy")
                for frame_ in np.arange(2):
                    for i in range(storey - 1, 0, -1):
                        shear_th = ""
                        for j in range(bay):
                            if frame_face == "abc":
                                latex, asd = self.beam_moment(
                                    force_list[frame_, i, j], bay_widths[j]
                                )
                                if frame_ == 0:
                                    frame = [chr(65), chr(65 + 2)]
                                else:
                                    frame = chr(65 + frame_)
                            else:
                                latex, asd = self.beam_moment1(
                                    force_list[frame_, i, j],
                                    bay_widths[j],
                                    slab_force[frame_, i, j],
                                )
                                if frame_ == 0:
                                    frame = [frame_ + 1, frame_ + 3]
                                else:
                                    frame = frame_ + 2
                            if i == 1:
                                i_ = [i + 1, i]
                            else:
                                i_ = i + 1
                            discription = (
                                f"Storey-{i_}-Bay-{j+1}-Frame-{frame}-Load-{load_type}"
                            )
                            name = (
                                f"beam_moment_latex_{load_type}_{frame}_{frame_face}"
                            )
                            self.add_latex(
                                key=name, values=latex, discription=discription
                            )
    def handcalc_approximate_and_figures(self):
        self.distributed_load_calculation()
        self.beam_mom_approximate()
        self.beam_sh_approximate()
        self.distributed_frame_display()
        self.beam_bending_moment_distributed()
        self.beam_shear_distributed()
        self.col_moment_distribution()
    def handcalc_portal_and_figures(self):
        self.portal_computations()
        self.portal_plotting()
        self.handcalc_portal()

    @handcalc(override=params,jupyter_display=jupyter)
    def computation_load_c(self,d,l ,e ):
        d = d 
        l = l 
        e = e 
        load_combo1 = d+l 
        load_combo2 = d +0.7*e
        load_combo3 = d +0.75*l+0.75*(0.7*e)


        return load_combo1,load_combo2,load_combo3
    def load_combination_beam(self):
        struc = self.no_frames
        storey = self.storey
        bay = self.bay
        m_end1 = np.zeros((2,struc,storey, bay))#Bending moment for beam
        m_mid1 = np.zeros((2,struc,storey, bay))#Bending moment for beam    
        sh_end1 = np.zeros((2,struc,storey, bay))#Bending moment for beam
        m_max1 = np.zeros((2,struc,storey, bay))#Bending moment for beam

        # m_end2 = np.zeros((struc,storey, bay))#Bending moment for beam
        # m_mid2 = np.zeros((struc,storey, bay))#Bending moment for beam    
        # sh_end2 = np.zeros((struc,storey, bay))#Bending moment for beam
        # m_max2 = np.zeros((struc,storey, bay))#Bending moment for beam
        frame_face = [1,2]
        shear_tr = ""
        load_combo_latex = ""
        for face_number,frame_face_ in enumerate(self.frame_face):
            for frame_ in [0,1]:
                for i in [1,2]:
                    for j in range(bay):
                                path1 = self.path_data
                                shear_th = ""
                                bse = (pd.read_csv(f"{path1}\\bsd-{frame_face_}.csv")).to_numpy()
                                bme = (pd.read_csv(f"{path1}\\bmt-{frame_face_}.csv")).to_numpy()
                                e_shear = bse[i,j]
                                e_mom = bme[i,j]

                                
                                bsd = (np.load(f"{path1}\\beams_shear1_Dead_{frame_face_}.npy"))
                                bmd = (np.load(f"{path1}\\beams_bending1_Dead_{frame_face_}.npy"))
                                bsl = (np.load(f"{path1}\\beams_shear1_Live_{frame_face_}.npy"))
                                bml = (np.load(f"{path1}\\beams_bending1_Live_{frame_face_}.npy"))
                                bmmd = (np.load(f"{path1}\\beams__mid_bending1_Dead_{frame_face_}.npy"))
                                bmml = (np.load(f"{path1}\\beams__mid_bending1_Live_{frame_face_}.npy"))

                                latex_mom,asd_mom = self.computation_load_c(d=bmd[frame_,i,j],l=bml[frame_,i,j],e=e_mom) 
                                latex_sh,asd_sh = self.computation_load_c(d=bsd[frame_,i,j],l =bsl[frame_,i,j],e =e_shear)
                                latex_mid,asd_mid = self.computation_load_c(d=bmmd[frame_,i,j],l =bmml[frame_,i,j],e =0)

                                
                                m_end1[face_number,frame_,i,j] = np.max(asd_mom)
                                m_mid1[face_number,frame_,i,j] = np.max(asd_mid)
                                    
                                moment1 = np.max([m_end1[face_number,frame_,i,j],m_mid1[face_number,frame_,i,j] ] )
                                shear1 = np.max(asd_sh)
                                m_max1[face_number,frame_,i,j] = moment1
                                sh_end1[face_number,frame_,i,j] = shear1                        
                    
                                if frame_== 0:
                                    if face_number == 0:
                                        girder_number = [(frame_*2) + j+1,((frame_+1)*2*2) + j+1]
                                        
                                    else:
                                        girder_number = [(frame_*2) + j+1 + 6,((frame_+2)*2) + j+1 + 6]
                                else:
                                    if face_number == 0:
                                        girder_number = [(frame_*2) + j+1]
                                        
                                    else:
                                        girder_number = [(frame_*2) + j+1 + 6]
                                if i == 1:
                                    stry = [1,2]
                                else:
                                    stry = [3]
                                self.add_latex(f'load_combo_{girder_number}_{stry}',latex_mom,f"Girder-{girder_number}-Storey-{stry}-End Moment")
                                self.add_latex(f'load_combo_{girder_number}_{stry}',latex_mid,f"Girder-{girder_number}-Storey-{stry}-Mid Moment")
                                self.add_latex(f'load_combo_{girder_number}_{stry}',latex_sh,f"Girder-{girder_number}-Storey-{stry}--Shear")
        path_data = self.path_data

        ret = np.save((f"{path_data}\\m_end1.npy"),m_end1)
        ret = np.save((f"{path_data}\\m_mid1.npy"),m_mid1)
        ret = np.save((f"{path_data}\\sh_end1.npy"),sh_end1)
        ret = np.save((f"{path_data}\\m_max1.npy"),m_max1)
    def load_combination_column(self):
        struc = self.no_frames
        storey = self.storey
        bay = self.bay
        col_mom = np.zeros((2,struc,storey, bay+1))#Bending moment for beam
        m_mid1 = np.zeros((2,struc,storey, bay))#Bending moment for beam    
        sh_end1 = np.zeros((2,struc,storey, bay))#Bending moment for beam
        m_max1 = np.zeros((2,struc,storey, bay))#Bending moment for beam
        col_axial = np.zeros((struc,storey, bay+1))
        frame_face = [1,2]
        shear_tr = ""
        load_combo_latex = ""
        path_data = self.path_data
        for face_number,frame_face_ in enumerate(self.frame_face):
            for frame_ in range(struc):
                for i in [0,2]:
                    for j in range(bay+1):
                                if frame_face_ == "abc":
                                    cae = (pd.read_csv(f"{path_data}\\axial_list-{frame_face_}.csv")).to_numpy()
                                    beam_shear_dead = np.load(f"{path_data}\\beams_shear1_Dead_{frame_face_}.npy")
                                    beam_shear_live = np.load(f"{path_data}\\beams_shear1_Live_{frame_face_}.npy")
                                    if j  == 0:
                                        cad = beam_shear_dead[frame_,i,j]
                                        cal = beam_shear_live[frame_,i,j]
                                        # col_axial[frame_,i,j]  = beam_shear_dead[frame_,i,j]
                                    elif j == 1:
                                        cad = beam_shear_dead[frame_,i,j-1] + beam_shear_dead[frame_,i,j]
                                        cal = beam_shear_live[frame_,i,j-1] + beam_shear_dead[frame_,i,j]
                                        # col_axial[frame_,i,j] = beam_shear[frame_,i,j] + beam_shear[frame_,i,j-1]
                                    elif j == 2:
                                        cad = beam_shear_dead[frame_,i,j-1]
                                        cal = beam_shear_live[frame_,i,j-1]
                                        # col_axial[frame_,i,j] = beam_shear[frame_,i,j-1]
                                    e_axial = cae[i,j]
                                    latex_axial,asd_axial = self.computation_load_c(d=cad,l=cal,e=e_axial) 
                                    col_axial[frame_,i,j] = max(asd_axial)
                                    # beam_shear = np.load(f"{path_data}\\beams_shear1_{load_type_}_abc.npy")



                                    if frame_face_ == "abc":
                                        frame_name = chr(65+frame_)
                                    else:
                                        frame_name = frame_+ 1
                                    shear_th = ""
                                    path = self.path_data
                                    cse = (pd.read_csv(f"{path}\\csf-{frame_face_}.csv")).to_numpy()
                                    cme = (pd.read_csv(f"{path}\\cmt-{frame_face_}.csv")).to_numpy()
                                    cmd = np.load(f"{path}\\col_moment1_Dead_2_{frame_face_}.npy")
                                    cml = np.load(f"{path}\\col_moment1_Live_2_{frame_face_}.npy")
                                    e_shear = cse[i,j]
                                    e_mom = cme[i,j]
                                    
                                    latex_mom,asd_mom = self.computation_load_c(d=cmd[i,j],l=cml[i,j],e=e_mom) 
                                    # latex_sh,asd_sh = self.computation_load_c(d=bsd[i,j],l =bsl[i,j],e =e_shear)
                                    col_mom[face_number,frame_,i,j] = max(asd_mom)
                                    self.add_latex(f'load_combo_column_{frame_}_{frame_face_}_{j+1}_{i+1}',latex_mom,f"Column-{j+1}-Storey-{i+1}-Moment-Frame {chr(65+frame_)}") 
                                    self.add_latex(f'load_combo_column_{frame_}_{frame_face_}_{j+1}_{i+1}',latex_axial,f"Column-{j+1}-Storey-{i+1}-Axial-Frame {chr(65+frame_)}")  
        path_data = self.path_data                     
        ret = np.save((f"{path_data}\\col_mom.npy"),col_mom)
        ret = np.save((f"{path_data}\\col_axial.npy"),col_axial)
    def test(self):
        struc = self.no_frames
        storey = self.storey
        bay = self.bay
        m_end1 = np.zeros((2,struc,storey, bay))#Bending moment for beam
        m_mid1 = np.zeros((2,struc,storey, bay))#Bending moment for beam    
        sh_end1 = np.zeros((2,struc,storey, bay))#Bending moment for beam
        m_max1 = np.zeros((2,struc,storey, bay))#Bending moment for beam

        m_end2 = np.zeros((struc,storey, bay))#Bending moment for beam
        m_mid2 = np.zeros((struc,storey, bay))#Bending moment for beam    
        sh_end2 = np.zeros((struc,storey, bay))#Bending moment for beam
        m_max2 = np.zeros((struc,storey, bay))#Bending moment for beam
        frame_face = [1,2]
        shear_tr = ""
        load_combo_latex = ""
        for face_number,frame_face_ in enumerate(self.frame_face):
            for frame_ in [0,1]:
                for i in [0,2]:
                    for j in range(bay):
                                path1 = self.path_data
                                shear_th = ""
                                bse = (pd.read_csv(f"{path1}\\bsd-{frame_face_}.csv")).to_numpy()
                                bme = (pd.read_csv(f"{path1}\\bmt-{frame_face_}.csv")).to_numpy()
                                e_shear = bse[i,j]
                                e_mom = bme[i,j]

                                
                                bsd = (np.load(f"{path1}\\beams_shear1_Dead_{frame_face_}.npy"))
                                bmd = (np.load(f"{path1}\\beams__mid_bending1_Dead_{frame_face_}.npy"))
                                bsl = (np.load(f"{path1}\\beams_shear1_Live_{frame_face_}.npy"))
                                bml = (np.load(f"{path1}\\beams__mid_bending1_Live_{frame_face_}.npy"))
                                bmmd = (np.load(f"{path1}\\beams__mid_bending1_Dead_{frame_face_}.npy"))
                                bmml = (np.load(f"{path1}\\beams__mid_bending1_Live_{frame_face_}.npy"))
             # print(bsl)
        # print(bml)
                                # print(bmmd)
                                # print(bmml)

    ##############################################################################################################################################################
    @handcalc(override="short",jupyter_display=jupyter)
    def Fv_Calc(self,f_y,m_u,d,tw,bf,tf,l_b,A,S_x,shear,I_x):
        V = shear
        Q = tf*bf*(d/2)
        if d/tw < 998/sqrt(f_y): F_v = (0.4*f_y)
        elif d/tw > 998/sqrt(f_y):F_v = (0.4*f_y)
        f_v = (V*Q)/(I_x*tw)
        if f_v<F_v: Shear = "Safe"
        if f_v>F_v: Shear = "Unsafe"
        return Shear

    @handcalc(override="short",jupyter_display=False)
    def Sx_required(self,f_y,m_u,d,tw,bf,tf,l_b,A,S_x):
        Sx_req = m_u/(0.66*f_y)
    @handcalc(override="long",jupyter_display=False)
    def compact_check(self,f_y,m_u,d,tw,bf,tf,l_b,A):
        if bf/(2*tf) < 170/sqrt(f_y): Flange = "Compact"
        elif bf/(2*tf) > 170/sqrt(f_y):Flange = "Non_Compact"
        if d/tw < 1680/sqrt(f_y): Web = "Compact"
        elif d/tw > 1680/sqrt(f_y):Web = "Non_Compact"
    # @handcalc(override="short",jupyter_display=False)
    # def case_check(self,f_y,m_u,d,tw,bf,tf,l_b,A,S_x):
    #     c_b = 1
    #     I = ((1/6 * d )*tw**3)/12 + ((bf**3)*tf)/12
    #     rt = sqrt(I/A) * mm
    #     SR = l_b/rt
    #     l_1 = min(sqrt((703270*c_b)/f_y),sqrt((3516330*c_b)/f_y)) 
    #     l_2 = max(sqrt((703270*c_b)/f_y),sqrt((3516330*c_b)/f_y))
    #     if l_b<l_1:case = 1
    #     elif l_2>l_b>l_1:case = 2
    #     elif l_b>l_2>l_1:case = 3
    #     return case
    @handcalc(override="short",jupyter_display=False)
    def slenderness_check(self,f_y,m_u,d,tw,bf,tf,l_b,A,S_x):
        c_b = 1
        I = ((1/6 * d )*tw**3)/12 + ((bf**3)*tf)/12
        rt = sqrt(I/A) * mm
        SR = l_b/rt
        return SR
    @handcalc(override="short",jupyter_display=False)
    def case_check(self,f_y,m_u,d,tw,bf,tf,l_b,A,S_x):

        l_c = min((200*bf)/sqrt(f_y),137900/((f_y*d)/(bf*tf)))
        l_u = max((200*bf)/sqrt(f_y),137900/((f_y*d)/(bf*tf)))
        if l_b<l_c:case =1
        elif l_u>l_b>l_c:case =2
        elif l_b>l_u>l_c:case =3
        return case
    @handcalc(override="short",jupyter_display=False)
    def fb_check_case3(self,fy,f_y,m_u,d,tw,bf,tf,l_b,A,S_x,SR,c_b):
        l_1 = min(sqrt((703270*c_b)/fy),sqrt((3516330*c_b)/fy)) 
        l_2 = max(sqrt((703270*c_b)/fy),sqrt((3516330*c_b)/fy)) 
        if l_2> SR > l_1:F_b = max(f_y*((2/3)-(f_y*SR**2)/(10.55*10**6*c_b)),(82740*c_b)/((l_b*d)/(bf*tf))) * MPa
        elif l_2< SR > l_1:F_b = max((1172100*c_b)/(SR**2),(82740*c_b)/((l_b*d)/(bf*tf))) * MPa
        f_b = m_u/S_x
        if f_b < F_b: Bending = "Safe"
        elif f_b > F_b: Bending = "Unsafe"
        return Bending
    @handcalc(override="short",jupyter_display=False)
    def fb_check(self,f_y,m_u,d,tw,bf,tf,l_b,A,S_x,case):
        if case == 1:F_b = 0.66* f_y 
            
        elif case == 2:F_b = 0.60* f_y 
            
        f_b = m_u/S_x
        if f_b < F_b: Bending = "Safe"
        elif f_b > F_b: Bending = "Unsafe"
        return Bending
    @handcalc(override="params",jupyter_display=False)
    def parameters(self,new,mu,fy,lb,shear,Sx_req,tolerance):
        name = new["AISC_Manual_Label"][tolerance] 
        A = new["A.1"][tolerance] * mm **2
        d = new["d.1"][tolerance] * mm
        b_f = new["bf.1"][tolerance]* mm
        t_f = new["tf.1"][tolerance]* mm
        t_w = new["tw.1"][tolerance]* mm
        S_x = new["Sx.1"][tolerance]  * mm **3
        S_y = new["Sy.1"][tolerance] * mm **3
        I_x = new["Ix.1"][tolerance]* mm **4
        I_y = new["Iy.1"][tolerance]* mm **4
        Sx_req = Sx_req
        m_u = mu 
        f_y = fy
        l_b = lb
        shear = shear

        return A,d,b_f,t_f,t_w,S_x,S_y,I_x,I_y,name
    @handcalc(override="params",jupyter_display=False)
    def parameters_column(self,new,mu,fy,axial,tolerance):

        name = new["AISC_Manual_Label"][tolerance] 
        A = new["A.1"][tolerance] * mm **2
        d = new["d.1"][tolerance] * mm
        b_f = new["bf.1"][tolerance]* mm
        t_f = new["tf.1"][tolerance]* mm
        t_w = new["tw.1"][tolerance]* mm
        S_x = new["Sx.1"][tolerance] * mm **3
        S_y = new["Sy.1"][tolerance]* mm **3
        I_x = new["Ix.1"][tolerance]* mm **4
        I_y = new["Iy.1"][tolerance]* mm **4
        r_x = new["rx.1"][tolerance]* mm
        r_y = new["ry.1"][tolerance]* mm
        m_u = mu 
        f_y = fy
        axial = axial

        return A,d,b_f,t_f,t_w,S_x,S_y,I_x,I_y,name,r_x,r_y
    def beam_parameters(self):
        path_data = self.path_data
        excel = pd.read_excel("1.xlsx",sheet_name = 1,mangle_dupe_cols=True)
        new = pd.DataFrame(excel["AISC_Manual_Label.1"])
        l = ["AISC_Manual_Label","A.1","d.1","W.1","rx.1","ry.1","Sx.1","Sy.1","bf.1","tf.1","tw.1","Ix.1","Iy.1"]
        for x in l:
            new[f"{x}"] = (excel[f"{x}"])
        section = "W"
        section1 = "T"
        mask = new["AISC_Manual_Label.1"].str.startswith(section, na=False)

        new = new[mask].copy().reset_index()
        mask = ~new["AISC_Manual_Label.1"].str.contains(section1, na=False)
        new = new[mask].copy().reset_index()
        new.to_csv(f"w_sections.csv")
        # Area = new["A.1"][0] * mm **2
        # d = new["d.1"][0] * mm
        # b_f = new["bf.1"][0] * mm
        # t_f = new["tf.1"][0] * mm
        # t_w = new["tw.1"][0] * mm 
        # S_x = new["Sx.1"][0] *10**3 
        # S_y = new["Sy.1"][0] *10**3 
        # I_x = new["Ix.1"][0]*10**6 
        # I_y = new["Iy.1"][0]*10**6 
        new["Sx.1"] = new["Sx.1"] *10**3 
        new["Sy.1"] = new["Sy.1"] *10**3 
        new["Ix.1"] = new["Ix.1"]*10**6 
        new["Iy.1"] = new["Iy.1"]*10**6 
        new.to_csv(f"{path_data}\\new1.csv")
        path_data = self.path_data
        # m_end1 = np.load(f"{path_data}\\m_end1.npy")
        # m_mid1 = np.load(f"{path_data}\\m_mid1.npy")
        sh_end1 = np.load(f"{path_data}\\sh_end1.npy")
        m_max1 = np.load(f"{path_data}\\m_max1.npy")
        m_u = np.max(m_max1) * kN * m
        # m_u = 200* kN * m
        m_u_calc = np.max(m_max1) * 10**6
        # m_u_calc = 200 * 10**6
        print(np.max(m_max1))
        # m_u_calc = 215 * 10**6
        shear = np.max(sh_end1) * kN 
        f_y_calc = self.f_y 
        fy = self.f_y 
        f_y = self.f_y *MPa
        Sx_req = (m_u_calc/(0.66*f_y_calc))
        print(Sx_req)
        new = new[new["Sx.1"]>Sx_req]

        new = new.sort_values(['A.1'], ascending = True)
        new = new.reset_index(drop=True)
        # new["A.1"] = new["A.1"] * mm **2
        # new["d.1"] = new["d.1"] * mm
        # new["bf.1"] = new["bf.1"] * mm
        # new["tf.1"] = new["tf.1"] * mm
        # new["tw.1"]  = new["tw.1"] * mm 
        # new["Sx.1"] = new["Sx.1"] * mm **3
        # new["Sy.1"] = new["Sy.1"] * mm **3
        # new["Ix.1"] = new["Ix.1"]* mm **4
        # new["Iy.1"] = new["Iy.1"] *mm **4


        new.to_csv(f"{path_data}\\new.csv")
        lb = 7000*mm
        # for tolerance in range(20):
        for tolerance in range(len(new.index)):
            if len(self.name_beams) < 6:
            
                try:
                    latex_params,asd= self.parameters(new,m_u,f_y,lb,shear,Sx_req,tolerance=tolerance)
                    Area,d,b_f,t_f,t_w,S_x,S_y,I_x,I_y,name_beam = asd
                    
                    latex_sxreq,asd = self.Sx_required(f_y,m_u,d,t_w,b_f,t_f,lb,Area,S_x)
                    latex_slenderness,SR = self.slenderness_check(fy,m_u,d,t_w,b_f,t_f,lb,Area,S_x)
                    latex_compact,asd = self.compact_check(fy,m_u,d,t_w,b_f,t_f,lb,Area)
            
                    latex_case_check,case = self.case_check(fy,m_u,d,t_w,b_f,t_f,lb,Area,S_x)

                    if case == 3:
                        latex_bending_check,bending = self.fb_check_case3(fy,f_y,m_u,d,t_w,b_f,t_f,lb,Area,S_x,SR,1)
                    else:
                        latex_bending_check,bending = self.fb_check(f_y,m_u,d,t_w,b_f,t_f,lb,Area,S_x,case)
                    latex_shear, Shear = self.Fv_Calc(f_y,m_u,d,t_w,b_f,t_f,lb,Area,S_x,shear,I_x)
                    # self.add_latex(f'test',latex_sxreq,f"Sx Requirement")
                    if bending == "Safe" and Shear == "Safe":
                    # self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_sxreq+latex_params,f"Try {name_beam} (Parameters Below)")
                    # self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_slenderness,f"Slenderness Check")
                    # self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_compact,f"Compact Check")
                    # self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_case_check,f"Bending Check")
                    # self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_bending_check,f"Bending Check Continuation")
                    # self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_shear,f"Shear Check")
                    # self.name_beams.append(name_beam)
                        self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_sxreq+latex_params,f"Try {name_beam} (Parameters Below)")
                        self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_slenderness,f"Slenderness Check")
                        self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_compact,f"Compact Check")
                        self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_case_check,f"Bending Check")
                        self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_bending_check,f"Bending Check Continuation")
                        self.add_latex(f'BEAM ANALYSIS CALCULATION_{name_beam}',latex_shear,f"Shear Check")
                        self.name_beams.append(name_beam)
                except:
                    pass

        return new
    @handcalc(override="symbolic",jupyter_display=False)
    def analysis_col_assume(self,f_a =1 ,F_a =1,f_b =1,F_b =1,K = 1,l = 1,r =1 ):
        assume = f_a/F_a <= 0.5
        assume = f_b/F_b <= 0.5
        assume = K*l/r == 0
    @handcalc(override="short",jupyter_display=False)
    def analysis_col_assume1(self,p,m_x,f_a =1 ,F_a =1,f_b =1,F_b =1,K = 0,l = 0,r =0,C_c = 0,F_y = 248 * MPa):
        F_s = 5/3 
        F_a = (1 - 0)*(F_y/F_s)
        F_a_ = 0.8*F_a
        f_a = F_a_*0.5
        Area_req = p/f_a
        F_b = 0.6*F_y
        f_b = F_b*0.8
        Sx_req = m_x/f_b
        return Area_req,Sx_req
    @handcalc(override="short",jupyter_display=False)
    def k_finder(self,I_x,Ix_beam,col_name,beam_name):
        col_section = col_name
        beam_section = beam_name
        column_1 = (I_x)/(4000*mm)
        column_2 = (I_x)/(3000*mm)
        beam_1 = (Ix_beam)/(7000*mm)
        beam_2 = (Ix_beam)/(6000*mm)
        G_d = (column_1+column_2)/(beam_1+beam_2)
        G_a = 1.0 # Fixed
        return G_d,G_a
    # @handcalc(override="short",jupyter_display=False)
    def k_solver(self,G_d,G_a):
        for k in np.arange(1.5,2.5,0.05):
            x = (((G_d*G_a*(pi/k)**2)-36)/(6*(G_d+G_a) )- ((pi/k)/tan(pi/k)))
            if (x < 0.1) and (x > 0):
                break
        return k,x
    @handcalc(override="short",jupyter_display=False)
    def k_solver_display(self,k,G_d,G_a):
        k = k#From Alignment Chart
        x = (((G_d*G_a*(pi/k)**2)-36)/(6*(G_d+G_a) )- ((pi/k)/tan(pi/k)))
        return x
    @handcalc(override="short",jupyter_display=False)
    def slenderness_col_check(self,k,l,r_x,r_y,f_y):
        SR = max((k*l)/r_x,(k*l)/r_y)
        C_c = sqrt((2*pi**2*200000*MPa)/f_y)
        if SR<C_c: Column = "Short"
        elif SR>C_c: Column = "Slender"
        return Column,C_c,SR
    @handcalc(override="short",jupyter_display=False)
    def Fa_col(self,SR,C_c,f_y,axial,A,E_s,m_x,S_x,b_f,d,t_f,l_b,fy,Cm = 0.85):    
        F_s = 5/3 + (3*(SR)/(8*C_c)) - ((SR)**3)/(8*C_c**3)
        F_a = (1 - (SR**2)/(2*C_c)**2)*(f_y/F_s)
        f_a = axial/A
        Fe_x = (12*pi**2*E_s)/(23*(SR)**2)
        Magnification_factor = Cm/(1-(f_a/Fe_x))
        if Magnification_factor < 1: Magnification_factor = Magnification_factor
        elif Magnification_factor > 1: Magnification_factor = 1
        f_bx = m_x/S_x
        l_c = min((200*b_f)/sqrt(fy),137900/((fy*d)/(b_f*t_f)))
        l_u = max((200*b_f)/sqrt(fy),137900/((fy*d)/(b_f*t_f)))
        if l_b<l_c:case =1;F_bx = 0.66*f_y
        elif l_u>l_b>l_c:case =2;F_bx = 0.60*f_y
        Stability = f_a/F_a + Magnification_factor*(f_bx/F_bx)
        if Stability < 1: Column = "Safe"
        elif Stability > 1: Column = "Unsafe"
        Strenght  = (f_a/(0.60*f_y)) + (f_bx/F_bx)
        if Strenght < 1: Column = "Safe"
        elif Strenght > 1: Column = "Unsafe"
        
    def column_parameters(self):
        excel = pd.read_excel("1.xlsx",sheet_name = 1,mangle_dupe_cols=True)
        new = pd.DataFrame(excel["AISC_Manual_Label.1"])
        l = ["AISC_Manual_Label","A.1","d.1","W.1","rx.1","ry.1","Sx.1","Sy.1","bf.1","tf.1","tw.1","Ix.1","Iy.1"]
        for x in l:
            new[f"{x}"] = (excel[f"{x}"])
        section = "W"
        section1 = "T"
        mask = new["AISC_Manual_Label.1"].str.startswith(section, na=False)
        new = new[mask].copy().reset_index(drop=True)
        mask = ~new["AISC_Manual_Label.1"].str.contains(section1, na=False)
        new = new[mask].copy().reset_index(drop=True)
        new = new.sort_values(['A.1'], ascending = True)
        new = new.reset_index(drop=True)
        new["Sx.1"] = new["Sx.1"] *10**3 
        new["Sy.1"] = new["Sy.1"] *10**3 
        new["Ix.1"] = new["Ix.1"]*10**6 
        new["Iy.1"] = new["Iy.1"]*10**6 
        new.to_csv("mab.csv")
        path_data = self.path_data
        new_name = new.set_index('AISC_Manual_Label')
        col_mom = np.load(f"{path_data}\\col_mom.npy")
        col_axial = np.load(f"{path_data}\\col_axial.npy")
        mu = np.max(col_mom) * kN*m
        axial = np.max(col_axial) * kN
        f_y_calc = self.f_y 
        fy = self.f_y 
        f_y = self.f_y *MPa
        E_s = 200000*MPa
        l = 4000* mm

        # new1 = new1.iloc[0]
        latex_assume1,asd = self.analysis_col_assume1(axial,mu)
        area_req,sx_req = asd
                    
        new_default = new[(new["Sx.1"]>sx_req/(mm**3))]
        new_default = new_default.sort_values(['d.1'], ascending = True)
        new_default = new_default.reset_index(drop=True)        
        for tolerance in range(len(new_default)):
                for name_beams_ in self.name_beams:
                    if len(self.column_counter)<6:
                        Ix_beam = new_name["Ix.1"][name_beams_] * mm**4
                        latex_parameters_col,asd = self.parameters_column(new_default,mu,f_y,axial,tolerance)
                        A,d,b_f,t_f,t_w,S_x,S_y,I_x,I_y,name,r_x,r_y = asd
                        latex_k_finder,asd = self.k_finder(I_x,Ix_beam,name,name_beams_)
                        G_d,G_a = asd
                        k,verify_k = self.k_solver(G_d,G_a)
                        if 0 < verify_k < 0.1:
                            latex_k_display,asd = self.k_solver_display(k,G_d,G_a )
                            latex_slender_col,asd = self.slenderness_col_check(k,l,r_x,r_y,f_y)
                            slender_result,C_c,SR =asd
                            if slender_result == "Short" :
                                latex_Fa,asd = self.Fa_col(SR,C_c,f_y,axial,A,E_s,mu,S_x,b_f,d,t_f,l,fy,Cm = 0.85)
                                self.add_latex1(f'Column_{name_beams_}_{name}',latex_assume1+latex_parameters_col,f"Choose Section")
                                self.add_latex1(f'Column_{name_beams_}_{name}',latex_k_finder+latex_k_display+latex_slender_col,f"Check Slenderness")
                                self.add_latex1(f'Column_{name_beams_}_{name}',latex_Fa,f"Stability and Strength Check")
                                if name not in self.column_counter:
                                    self.column_counter.append(name) 
                    # except:
            #     pass


    def selenium_screenshot(self):
        desired_dpi =1.0
        chrome_options = Options()
        

        chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"--force-device-scale-factor={desired_dpi}")
        driver = webdriver.Chrome(r"chromedriver.exe",options=chrome_options)
        path = self.path_data
        print(path)
        for link_number,links in enumerate([f"{path}\\All_Calculations.html",f"{path}\\All_Calculations1.html"]):
            driver.get(links)
            time.sleep(5)
            driver.maximize_window()
            if link_number == 0:
                data = self.data
            elif link_number == 1:
                data = self.data1
            for keys in data.keys():
                path_latex = self.path_latex_comp
                element = driver.find_element(By.ID, keys)
                driver.execute_script("arguments[0].scrollIntoView(true);", element)
                element.screenshot(f"{path_latex}\\{keys}.png")
height = [4, 3, 3]  # Floor height
storey = 3
bay = 2
bay_widths = np.array([6, 5.5])
bay_widths_123 = np.array([7, 6])
# force_list = np.array([28.01, 49.02, 47.49])#Kerl
# force_list = np.array([30.17, 52.80, 52.93])#Jenny
force_list = np.array([21.88, 38.29, 40.96])#Pihuts
bay_widths_cm = np.hstack((0, bay_widths.cumsum()))
heights = height * np.ones(storey)
heights_cm = np.hstack((0, heights.cumsum()))
h_line = storey * bay
v_line = (bay + 1) * storey
color_list = ["darkseagreen"]
# for color_ in color_list:
hl = portalmethod(
        Height=height,
        Storey=storey,
        Bay=bay,
        Bay_abc=bay_widths,
        Bay_123=bay_widths_123,
        Force_list=force_list,
        No_frames=3,
        folder_name=f"Pihuts",
        fy = 248,
        color = f"cyan"
    )

hl.handcalc_approximate_and_figures()
hl.handcalc_portal_and_figures()
hl.load_combination_beam()
hl.load_combination_column()
hl.beam_parameters()
hl.column_parameters()
hl.html_makerl()
hl.html_makerl1()
hl.selenium_screenshot()



In [ ]:
print(len(hl.column_counter))

In [ ]:
test = np.load(r"C:\Users\63976\Documents\Plate_4_Steel\kerl\Data\col_axial.npy")
print(test)
np.argmax(test)

In [ ]:
@handcalc(override="short",jupyter_display=True)
def computation_load_c(d,l ,e ):
        d = d 
        l = l 
        e = e 
        load_combo1 = d+l 
        load_combo2 = d +0.7*e
        load_combo3 = d -0.7*e
        load_combo4 = d +0.75*l+0.75*(0.7*e)
        load_combo5  = d+0.75*l- 0.75*(0.7*e)         
col_axial = np.zeros((3,3, 2+1))
for frame_face_ in ["abc",123]:
            for frame_ in range(3):
                for i in [0,2]:
                    for j in range(bay+1):
                            for load_type_ in ["Dead","Live"]:
                                if frame_face_ == "abc":
                                    cae = (pd.read_csv(r"C:\Users\63976\Documents\Plate_4_Steel\kerl\Data\axial_list-abc.csv")).to_numpy()
                                    beam_shear_dead = np.load(r"C:\Users\63976\Documents\Plate_4_Steel\kerl\Data\beams_shear1_Dead_abc.npy")
                                    beam_shear_live = np.load(r"C:\Users\63976\Documents\Plate_4_Steel\kerl\Data\beams_shear1_Live_abc.npy")
                                    if j  == 0:
                                        cad = beam_shear_dead[frame_,i,j]
                                        cal = beam_shear_live[frame_,i,j]
                                        # col_axial[frame_,i,j]  = beam_shear_dead[frame_,i,j]
                                    elif j == 1:
                                        cad = beam_shear_dead[frame_,i,j-1] + beam_shear_dead[frame_,i,j]
                                        cal = beam_shear_live[frame_,i,j-1] + beam_shear_dead[frame_,i,j]
                                        # col_axial[frame_,i,j] = beam_shear[frame_,i,j] + beam_shear[frame_,i,j-1]
                                    elif j == 2:
                                        cad = beam_shear_dead[frame_,i,j-1]
                                        cal = beam_shear_live[frame_,i,j-1]
                                        # col_axial[frame_,i,j] = beam_shear[frame_,i,j-1]
                                    e_axial = cae[i,j]
                                    print(cad)
                                    latex_axial,asd_axial = computation_load_c(d=cad,l=cal[frame_,i,j],e=e_axial) 
                                    # col_axial[face_number,frame_,i,j] = max(asd_axial)

In [ ]:
import handcalcs.render
from handcalcs.decorator import handcalc
from IPython.display import HTML, display, Math, Latex, SVG
@handcalc(override="short",jupyter_display=False)
def sumsd(a=1,b=1):
    if a>10: d = 10
    elif a<10: d = "mabuhay ka"
    c = a+b
    return c

@handcalc(override="short",jupyter_display=False)
def divide(c):
    c = c/2
    return c
# @handcalc(override="short",jupyter_display=True)
def sumsd1():
    latex,c = sumsd(a=1,b=1)
    latex1,c1 = divide(c)
    latex_t = "panalo"+latex+"patalo"+latex1
    display(Latex(latex_t))

# sumsd1()

In [ ]:
col_axial = np.load(r"C:\Users\63976\Documents\Plate_4_Steel\Pihuts\Data\col_axial.npy")
print(col_axial)

In [ ]:
%%render symbolic
f_a =1 
F_a =1
f_b =1 
F_b =1
K = 1
assume = f_a/F_a <= 0.5
assume = f_b/F_b <= 0.5
assume = K*l/r == 0


In [ ]:
from sympy.solvers import solve
from math import cos,tan, pi
from sympy import symbols
G_a,G_b,k = symbols(["G_a","G_b","k"])
G_a = 0.21
G_b = 0.4
# @handcalc(override="short",jupyter_display=True)
def try_me():
    for k in np.arange(0.5,2,0.01):
        x = (((0.21*0.4*(pi/k)**2)-36)/(6*(0.21+0.4) )- ((pi/k)/tan(pi/k)))
        if (x < 0.1) and (x > 0):
            break
    return k 

k = try_me()
print(k)
# x = scipy.optimize.fsolve(try_me,1)
@handcalc(override="short",jupyter_display=True)
def divide():
        k = 1.95
        x = (((0.21*0.4*(pi/k)**2)-36)/(6*(0.21+0.4) )- ((pi/k)/tan(pi/k)))

divide()